# 제품 이상여부 판별 프로젝트


## hard voting (csv 파일 이용)

모델을 통해 예측해 낸 정답지(csv) 파일의 결과값을 불러와 voting 하는 방식  
이때의 voting은 hard방식만 가능(예측확률은 저장되어있지 않음으로 soft 보팅은 불가능)  

In [1]:
import pandas as pd
import numpy as np
from collections import Counter

def read_submission_files(file_paths):
    """
    제출 파일을 읽어와서 예측 결과를 반환합니다.

    Parameters:
    file_paths (list of str): 제출 파일 경로 리스트

    Returns:
    list of np.array: 각 제출 파일의 예측 결과 리스트
    """
    predictions = []
    for file_path in file_paths:
        df = pd.read_csv(file_path)
        preds = df['target'].apply(lambda x: 1 if x == 'AbNormal' else 0).values
        predictions.append(preds)
    return predictions

def hard_voting(preds):
    """
    하드 보팅을 사용하여 최종 예측을 수행합니다.

    Parameters:
    preds (list of np.array): 각 모델의 예측 배열 리스트

    Returns:
    np.array: 최종 예측 결과
    """
    preds = np.array(preds)
    
    # 각 샘플의 예측 결과를 문자열로 변환하여 리스트에 저장
    sample_predictions = [''.join(map(str, x)) for x in preds.T]
    
    # 각 예측 결과의 빈도수를 계산
    prediction_counts = Counter(sample_predictions)
    
    # 빈도수 출력
    for pred, count in prediction_counts.items():
        print(f"Prediction {pred}: {count} times")
    
    # 하드 보팅을 통해 최종 예측을 계산
    final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=preds)
    return final_predictions

submission_file 에서 csv 파일을 불러와 hard voting 하는 알고리즘  

In [3]:
import pandas as pd

# 공통 경로
common_path = "../../submission_file/"

# 제출 파일 이름 리스트
file_names = [
    "data0817_light_all(1+4)_process_cutoff0.24_submission.csv"
    , "data0817_xgb_all(1+4)_process_cutoff0.24_submission.csv"
    , "data0817_cat_all(1+4)_process_cutoff0.3_submission.csv"
    # , 'data0817_rf_all(1+4)_process_cutoff0.24_submission.csv'
    # 파일 추가 가능  <----- 파일 필요시 추가하세요!!
]

# 경로를 추가하는 함수
def add_common_path(file_names, common_path):
    return [common_path + file_name for file_name in file_names]

# 경로가 추가된 파일 리스트
file_paths = add_common_path(file_names, common_path)

# 제출 파일에서 예측 결과 읽어오기
predictions = read_submission_files(file_paths)

# 하드 보팅 결과
final_predictions_hard = hard_voting(predictions)

# 결과를 새로운 제출 파일로 저장할 파일 이름
output_file_name = "final_submission.csv" # <----- 파일 이름을 변경하세요!!

# 결과를 새로운 제출 파일로 저장
df_sub = pd.read_csv(file_paths[0])
df_sub["target"] = final_predictions_hard
df_sub['target'] = df_sub['target'].apply(lambda x: 'AbNormal' if x == 1 else 'Normal')
df_sub.to_csv(output_file_name, index=False)

print(f"최종 제출 파일이 '{output_file_name}'로 저장되었습니다.")

Prediction 000: 16666 times
Prediction 111: 262 times
Prediction 010: 88 times
Prediction 100: 86 times
Prediction 110: 103 times
Prediction 101: 47 times
Prediction 001: 92 times
Prediction 011: 17 times
최종 제출 파일이 'final_submission.csv'로 저장되었습니다.


In [4]:
df_sub['target'].value_counts()

Normal      16932
AbNormal      429
Name: target, dtype: int64

In [5]:
df_sub.head(10)

,Set ID,target
0,0001be084fbc4aaa9d921f39e595961b,Normal
1,0005bbd180064abd99e63f9ed3e1ac80,Normal
2,000948934c4140d883d670adcb609584,Normal
3,000a6bfd02874c6296dc7b2e9c5678a7,Normal
4,0018e78ce91343678716e2ea27a51c95,Normal
5,001fda4596f545d0a3b0ce85fbea77d2,Normal
6,0020734a7b29472298358ad58645a0c9,Normal
7,00234c5914cd4c4a888d13f8b3773135,Normal
8,00297b6c93e44d49ac534758a23dc74e,AbNormal
9,002d904240d84b188d410d16383a9c3a,Normal
